In [1]:
# declare packages that you will use (pretty similar to LaTeX preamble)

# import Pkg

# Pkg.add("JuMP")

# Pkg.add("Ipopt")

using JuMP, Ipopt

# # compile functions you will be using later

# include("datagen.jl")

# include("jumpMLE.jl")

# # evaluate the functions referenced above in the -include- statements.
# # -@time- is equivalent to tic/toc in Matlab
# @time X,Y,bAns,sigAns,n = datagen()
# @time bOpt,sOpt = jumpMLE(Y,X,[bAns,sigAns])


In [2]:
function jumpMLE(Y,X,startval)
myMLE = Model(solver=IpoptSolver(tol=1e-8))
@variable(myMLE, b[i=1:size(X,2)], start = startval[i])
@variable(myMLE, s >=0.0, start = startval[end])
# Write your objective function
@NLobjective(myMLE, Max, (n/2)*log(1/(2*pi*s^2))-
sum((Y[i]-sum(X[i,k]*b[k], k=1:size(X,2)))^2, i=1:size(X,1))/(2s^2))
# Solve the objective function
status = solve(myMLE)
# Save estimates
bOpt = getValue(b[:])
sOpt = getValue(s)
return bOpt,sOpt
end


jumpMLE (generic function with 1 method)

In [3]:
function datagen()
## Generate data for a linear model to test optimization
rand(1234)
# N = convert(Int64,1e4)
# T = 5
# n = convert(Int64,N*T)
N = 1000
T = 5
n = N*T

# Generate the Xs
X = [ones(N*T,1) 5+3*randn(N*T,1) rand(N*T,1) 2.5+2*randn(N*T,1) 15+3*randn(N*T,1) .7-.1*randn(N*T,1)
5+3*randn(N*T,1) rand(N*T,1) 2.5+2*randn(N*T,1)
15+3*randn(N*T,1) .7-.1*randn(N*T,1) 5+3*randn(N*T,1)
rand(N*T,1) 2.5+2*randn(N*T,1) 15+3*randn(N*T,1)
.7-.1*randn(N*T,1)]

# generate the betas (X coefficients)
bAns = [ 2.15, 0.10,
0.50, 0.10, 0.75, 1.2,
0.10,
0.50, 0.10, 0.75, 1.2, 0.10,
0.50, 0.10,
0.75, 1.2 ]
# generate the std dev of the errors
sigAns = 0.3

# generate the Ys from the Xs, betas, and error draws
draw = 0 + sigAns*randn(N*T,1)
Y = X*bAns+draw

# return generated data so that other functions (below) have access
return X,Y,bAns,sigAns,n
end


datagen (generic function with 1 method)

In [4]:
@time X,Y,bAns,sigAns,n = datagen()
@time bOpt,sOpt = jumpMLE(Y,X,[bAns,sigAns])

LoadError: MethodError: no method matching +(::Int64, ::Matrix{Float64})
For element-wise addition, use broadcasting with dot syntax: scalar .+ array
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  +(::T, [91m::T[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +([91m::Union{MathOptInterface.ScalarAffineFunction{T}, MathOptInterface.ScalarQuadraticFunction{T}}[39m, ::T) where T at C:\Users\Regi Kusumaatmadja\.julia\packages\MathOptInterface\5WwpK\src\Utilities\functions.jl:1534
[0m  ...